Links to other notebooks in the same folder:
<a href='http://pivotal.io/data-science'><img src='https://raw.githubusercontent.com/crawles/Logos/master/Pivotal_TealOnWhite.png' width='200px' align='right'></a>

<nav class = "navbar navbar-light bg-faded">
    <ul class = "nav navbar-nav">
        <li class = "">
            <a class = "nav-link">notebook1</a>
        </li>
        <li class = "">
            <a class = "nav-link">notebook2</a>
        </li>
        <li class = "">
            <a class = "nav-link">notebook3</a>
        </li>
        

# Import useful libraries

In [1]:
from datetime import datetime
from dateutil.relativedelta import relativedelta
from IPython.core.display import display, HTML
import os
import sys
import urllib

# If we want to move the graph
# %matplotlib notebook
from IPython.core.magic import register_line_magic, register_cell_magic, register_line_cell_magic
from IPython.display import display
from IPython.display import HTML
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pandas.io.sql as psql
import psycopg2
import seaborn as sns

# Credentials file to connect to database
import credentials
from sql_functions import *

/Users/gregorytam/anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
# Changes logo to a Pivotal logo
jPrefs = urllib.urlopen("https://raw.githubusercontent.com/crawles/Logos/master/jupyterPrefs.js").read()
HTML('<script>{}</script>'.format(jPrefs))

In [3]:
# Set default cell width
display(HTML('<style>.container {width:80% !important;}</style>'))

# Set default matplotlib settings
mpl.rcParams['figure.figsize'] = (22, 8)
mpl.rcParams['lines.linewidth'] = 3
mpl.rcParams['figure.titlesize'] = 26
mpl.rcParams['axes.labelsize'] = 18
mpl.rcParams['axes.titlesize'] = 22
mpl.rcParams['xtick.labelsize'] = 14
mpl.rcParams['ytick.labelsize'] = 14
mpl.rcParams['legend.fontsize'] = 16

# Set Seaborn Colours
blue, green, red, purple, yellow, cyan = sns.color_palette()

# Connect to Database

In [4]:
conn = psycopg2.connect(database=credentials.database,
                        user=credentials.user,
                        password=credentials.password,
                        host=credentials.host
                       )
conn.autocommit = True

In [5]:
schema_name = 'template'
psql.execute('SET search_path TO {}'.format(schema_name), conn)

<cursor object at 0x11940f430; closed: 0>

# Define magic functions - helpful for interacting with cluster

In [6]:
@register_cell_magic
def readsql(line, cell):
    """
    Extract the code in the specific cell (should be valid SQL), 
    and execute it using the connection object to the backend 
    database. The resulting pandas DataFrame is rendered inline 
    below the cell using IPython.display. You'd use this for SELECT.
    
    Returns a DataFrame with the name specified in the magic
    function. If this is not specified, then the DataFrame
    is called _df.
    """
    # Use the global connection object defined above.
    global conn
    table_name = line
    # If there is more than one table name specified,
    # throw an exception.
    if len(table_name.split()) > 1:
        raise Exception('More than one table name specified')

    if len(table_name.split()) > 0:
        # If a table name is specified, store it as that
        globals()[table_name] = psql.read_sql(cell.format(**globals()), conn)
        display(globals()[table_name])

    else:
        # Otherwise, call it _df
        global _df
        _df = psql.read_sql(cell.format(**globals()), conn)
        display(_df)

    
@register_cell_magic
def execsql(line, cell):
    """
    Extract the code in the specific cell (should be valid SQL),
    and execute it using the connection object to the backend 
    database. You'd use this for CREATE/UPDATE/DELETE.
    """
    # Use the global connection object defined above.
    global conn
    psql.execute(cell.format(**globals()), conn)

# We delete these to avoid name conflicts for automagic to work
del execsql, readsql

# Autofill Table Names
One downside of interacting with a remote database in Python is that table names will not be imported in. When this is run, it will import all of the schema names as classes and their respective table names as variables. That way, when we type a schema name, we can use tab completion to list out all of its columns.

In [7]:
sql = '''
SELECT table_schema, array_agg(table_name::TEXT) AS tables
  FROM information_schema.tables
 GROUP BY table_schema;
'''
info_df = psql.read_sql(sql, conn)

class Schema:
    def __init__(self, tables):
        for t in tables:
            exec('self.{t} = "{t}"'.format(t=t))

for row in info_df.iterrows():
    command = '{s} = Schema({tables})'.format(s=row[1][0], tables=row[1][1])
    exec(command)

# Examples

## Create Tables
Here we can create tables in SQL simply by using a SQL command and putting <code>%%execsql</code> at the very top of the cell.

In [8]:
%%execsql
DROP TABLE IF EXISTS example_data_table;
CREATE TABLE example_data_table
   AS SELECT 1 AS col1,
             2 AS col2, 
             3 AS col3;
            
DROP TABLE IF EXISTS other_table;
CREATE TABLE other_table
   AS SELECT 1;

Now we can check how autocomplete works:

<img src='autofill.png' width= 300px align='left'>

Try it by typing "<code>template.</code>" below then pressing tab

In [ ]:
template.

## Viewing a Table
We can now view one of the tables that we just created.

In [9]:
%%readsql
SELECT *
  FROM example_data_table;

,col1,col2,col3
0,1,2,3


By default, this will store the resulting pandas DataFrame into a variable called <code>_df</code>.

In [10]:
_df

,col1,col2,col3
0,1,2,3


## Using Variable Table Names

We can also use variable table or column names in our commands.

In [11]:
table_name = 'example_data_table'

In [12]:
%%readsql
SELECT *
  FROM {table_name};

,col1,col2,col3
0,1,2,3


# Writing to a Desired DataFrame

Instead of writing to the DataFrame <code>_df</code>, we may want to specify it ourselves. This can be achieved by putting the desired name of the table after <code>%%showsql</code>.

In [13]:
%%readsql input_df
SELECT *
  FROM {table_name}

,col1,col2,col3
0,1,2,3


In [14]:
input_df

,col1,col2,col3
0,1,2,3
